In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum


In [29]:
top_similar_df = pd.read_pickle("similarity_df.pkl")
allevents = top_similar_df['event'].unique()
display(top_similar_df.head(5))
apollo= top_similar_df[top_similar_df['event']=='Apollo_11']
display(apollo)

,name,plot,similarity,release_date,event,genres
23141,The Founding of a Republic,In 1945 after the end of the Second Sino-Japan...,0.604864,2009,Negotiations_to_end_apartheid_in_South_Africa,"[Chinese Movies, Drama, War film, World cinema]"
39420,The Deluge,The film is set in the 17th century during the...,0.572665,1974,Negotiations_to_end_apartheid_in_South_Africa,[War film]
6928,The Founding of a Party,"During the early 20th century, China is marked...",0.572192,2011,Negotiations_to_end_apartheid_in_South_Africa,"[Drama, Historical fiction]"
39032,7 Man Army,"During the Second Sino- Japanese war 20,000 Ja...",0.564577,1976,Negotiations_to_end_apartheid_in_South_Africa,"[Chinese Movies, Martial Arts Film]"
27370,The Final Inch,The film depicts the problems still occurring ...,0.559969,2009,Negotiations_to_end_apartheid_in_South_Africa,"[Documentary, Short Film]"


,name,plot,similarity,release_date,event,genres
35867,Titan A.E.,"In 3028, the Drej, a malevolent, energy-based...",0.494681,2000,Apollo_13,"[Action, Adventure, Animation, Family Film, Sc..."
35867,Titan A.E.,"In 3028, the Drej, a malevolent, energy-based...",0.457863,2000,Sputnik_2,"[Action, Adventure, Animation, Family Film, Sc..."


In [19]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import torch


display("all events ", allevents)
display(top_similar_df.head(2))

tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased')

def calculateSentiment(text):
    ## Referenced this medium article : https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f
    ## Used the https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english from Huggingface 
    
    chunksize = 512
    tokens = tokenizer.encode_plus(text, add_special_tokens=False,
                                return_tensors='pt')

    input_id_chunks = list(tokens['input_ids'][0].split(510))
    mask_chunks = list(tokens['attention_mask'][0].split(510))

    # loop through each chunk
    for i in range(len(input_id_chunks)):
        # add CLS and SEP tokens to input IDs
        input_id_chunks[i] = torch.cat([
            torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
        ])
        # add attention tokens to attention mask
        mask_chunks[i] = torch.cat([
            torch.tensor([1]), mask_chunks[i], torch.tensor([1])
        ])
        # get required padding length
        pad_len = chunksize - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_len)
            ])
            mask_chunks[i] = torch.cat([
                mask_chunks[i], torch.Tensor([0] * pad_len)
            ])

    # # check length of each tensor
    # for chunk in input_id_chunks:
    #     print(len(chunk))

    ##  Now the final step of placing our tensors back into the dictionary style format we had before.
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(mask_chunks)

    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attention_mask.int()
    }
    input_dict

    ## run the model
    outputs = model(**input_dict)
    probs = torch.nn.functional.softmax(outputs[0], dim=-1)
    probs = probs.mean(dim=0)
    winner = torch.argmax(probs).item()


    return (probs, winner)
        

'all events '

array(['Negotiations_to_end_apartheid_in_South_Africa', 'Nelson_Mandela',
       'Fall_of_the_Berlin_Wall', 'Stanislav_Petrov', 'Falklands_War',
       'Attempted_assassination_of_Ronald_Reagan#:~:text=On%20March%2030%2C%201981%2C%20President,engagement%20at%20the%20Washington%20Hilton.',
       'Iranian_Revolution#:~:text=Iranian%20people%20voted%20in%20a,it%20created%20throughout%20the%20world.',
       'Apollo_13', 'Apollo_11',
       'Death_and_state_funeral_of_Winston_Churchill',
       'Civil_Rights_Act_of_1964', 'Assassination_of_John_F._Kennedy',
       'Berlin_Wall', 'Vietnam_War', 'Great_Leap_Forward', 'Sputnik_2',
       'Hungarian_Revolution_of_1956', 'Warsaw_Pact',
       'Brown_v._Board_of_Education', 'Korean_War', 'Marshall_Plan',
       'Cold_War', 'Truman_Doctrine',
       'Atomic_bombings_of_Hiroshima_and_Nagasaki',
       'Attack_on_Pearl_Harbor'], dtype=object)

,name,plot,similarity,release_date,event,genres
23141,The Founding of a Republic,In 1945 after the end of the Second Sino-Japan...,0.604864,2009,Negotiations_to_end_apartheid_in_South_Africa,"[Chinese Movies, Drama, War film, World cinema]"
39420,The Deluge,The film is set in the 17th century during the...,0.572665,1974,Negotiations_to_end_apartheid_in_South_Africa,[War film]


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


KeyboardInterrupt: 

In [ ]:
display("original shape" , top_similar_df.shape) 
top_similar_df_less = top_similar_df.groupby('event').apply(
    lambda x: x.loc[
        x['similarity'] > 0.5 ] 
).reset_index(drop=True)

top_similar_df_less = top_similar_df.groupby('event').apply(
    lambda x: x.loc[
        x['similarity'] < 0.512 ] 
).reset_index(drop=True)

display("less shape" , top_similar_df_less.shape) 

# atomic_df = top_similar_df_less[top_similar_df_less['event'] == 'Atomic_bombings_of_Hiroshima_and_Nagasaki']
# berlin_df = top_similar_df_less[top_similar_df_less['event'] == 'Nelson_Mandela']
# display(atomic_df.shape)
# display(berlin_df.shape)

top_similar_df_less['probability'] = top_similar_df_less['plot'].apply(lambda x: calculateSentiment(x)[1])
# atomic_df['probability'] = atomic_df['plot'].apply(lambda x: calculateSentiment(x)[0])
# berlin_df['probability'] = berlin_df['plot'].apply(lambda x: calculateSentiment(x)[0])


'original shape'

(2500, 6)

'less shape'

(1576, 6)

KeyboardInterrupt: 

In [ ]:
top_similar_df_less.head(10)


,name,plot,similarity,release_date,event,genres,probability
0,The Dream is Alive,The movie includes scenes from numerous shuttl...,0.653769,1985,Apollo_11,"[Documentary, Short Film]",0
1,Apollo 13,"On July 20, 1969, veteran astronaut Jim Lovell...",0.681827,1995,Apollo_13,"[Adventure, Drama, History]",0
2,The Dream is Alive,The movie includes scenes from numerous shuttl...,0.661487,1985,Apollo_13,"[Documentary, Short Film]",0
3,JFK,The film's opening encompasses newsreel footag...,0.682642,1991,Assassination_of_John_F._Kennedy,"[Biography, Drama, History, Mystery, Thriller]",0
4,Hiroshima,The film opens in April 1945 with the death of...,0.685578,1995,Atomic_bombings_of_Hiroshima_and_Nagasaki,"[Drama, Japanese Movies, Period piece, Politic...",0
5,Countdown to Looking Glass,The film begins with a fictional broadcast fro...,0.656042,1984,Cold_War,"[Drama, Political thriller, War film]",0
6,We Were Soldiers,{{Plot}} A French unit is on patrol in Vietnam...,0.694403,2002,Korean_War,"[Action, ActionAdventure, Combat Films, Drama,...",0
7,Countdown to Looking Glass,The film begins with a fictional broadcast fro...,0.663181,1984,Korean_War,"[Drama, Political thriller, War film]",0
8,Heaven's Soldiers,The film begins with high-level military leade...,0.662264,2005,Korean_War,"[Action, Comedy, Science Fiction]",0
9,Countdown to Looking Glass,The film begins with a fictional broadcast fro...,0.697152,1984,Vietnam_War,"[Drama, Political thriller, War film]",0
